# the datetime module

time is weird. 

working with dates and timestamps is frequently required, and often tricky. there are countless issues that can crop up due to differing time formats, calendars, time zones. in addition there are detailed obscure definitions of the various time measures we use. so time intervals and date arithmetics are not trivial to compute from first principles.

- for instance, years differ in length, a year is not even an integer number of days long. 
- a calendar year is not an even number of weeks.
- the months, and the (financial) quarters in a year are not all equally long. and not an integer multiple of weeks.
- fiscal years differ from calendar years (and are location specific!).
- we might need to count events during a time period in number of working days (excluding weekends and holidays from the count). 
- given a datestamp, consider how you can find out what week number does that time stamp take place in? to calculate this from first principles we'd need to to know 
    + what weekday is the first day of the week? (typically either on sunday or monday)
    + was it in a leap year?
    + what timezone was the timestamp recorded and in which timezone are we counting the weeks? (imagine we are aggregating by week as observed in UTC and the time stamp above was recorded in china standard time, many hours ahead. then the timestamp may actually belong to the "next week".)

luckily python has modules that help with these formats and calculations. we just need to learn to use them. here we will demonstrate the use of the `datetime` module. there are two aspects at play here. the first is date and time formating, and the second is date and time arithmetic. the latter is impossible without the former. 

### a plea:
please, please, please, from now on, whenever and whereever you need to display a date, no matter what the context is, use the [iso format designed to minimize misunderstanding and confusion](https://www.iso.org/iso-8601-date-and-time-format.html): *YYYY-MM-DD*. 

this is the internationally recognized date format and therefore the best format, bar none, whose adoption will reduce confusion and errors. use it. design dates fields in forms this way. date your notes this way. set your computer's localisation format to this. expect dates in this format by default. shame those who won't, for their only defense is bigotry favouring their own familiarity which hides a callous indifference to the unforeseeable errors caused by date confusion.

like any globally conscious python uses that format.

# this weeks exercise:
read in the rodent inspection data. 

count the number of inspections per [iso-week](https://en.wikipedia.org/wiki/ISO_week_date). find the week with the greatest number of inspections. for that week, and that week only, count the inspections by weekday.


In [1]:
import datetime

In [2]:
a_datetime = datetime.datetime(2001, 3, 16)
print('a date time:', a_datetime)
print('or just a date:', a_datetime.date())

a date time: 2001-03-16 00:00:00
or just a date: 2001-03-16


In [3]:
# let us start with getting the current timestamp
a_datetime = datetime.datetime.now()
print(type(a_datetime))
print(a_datetime)

<class 'datetime.datetime'>
2018-09-30 16:02:29.528105


In [4]:
a_datetime = datetime.datetime.now()
# individual elements:
print('the current year', a_datetime.year)
print('the current month', a_datetime.month)
print('the current day', a_datetime.day)
print('the current hour', a_datetime.hour)
print('the current minute', a_datetime.minute)
print('the current second', a_datetime.second)
print('the current microsecond', a_datetime.microsecond)
print('the current weekday number', a_datetime.weekday())

the date is: 2018-09-30
the current year 2018
the current month 9
the current day 30
the current hour 16
the current minute 2
the current second 29
the current microsecond 534185
the current weekday number 6


In [17]:
# there is also a date object: 
today_date = datetime.date.today()
print('the date is:', today_date)
print('the year is:', today_date.year)
print('the month is:', today_date.month)
print('the day-of-month is:', today_date.day)
print('the day-of-week is:', today_date.weekday())
print('the iso-weekday is:', today_date.isoweekday())
print('the components are:', today_date.timetuple())
try:
    print('the year is:', today_date.hour)
except:
    print('a "date" object has no "hour" attribute (nor "minute", "second")'
dir(today_date)

the date is: 2018-09-30
the year is: 2018
the month is: 9
the day-of-month is: 30
the day-of-week is: 6
the iso-weekday is: 7
the components are: time.struct_time(tm_year=2018, tm_mon=9, tm_mday=30, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=6, tm_yday=273, tm_isdst=-1)
a "date" object has no "hour" attribute


['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'ctime',
 'day',
 'fromisoformat',
 'fromordinal',
 'fromtimestamp',
 'isocalendar',
 'isoformat',
 'isoweekday',
 'max',
 'min',
 'month',
 'replace',
 'resolution',
 'strftime',
 'timetuple',
 'today',
 'toordinal',
 'weekday',
 'year']

to get a formated string with time and date descriptors. need a command that reformats time strings. the `strftime()` function does this (string - format - time).

In [5]:
print('today is', a_datetime.strftime("%a"), '(Weekday, short version)')
print('today is', a_datetime.strftime("%A"), '(Weekday, full version)')
print('the weekday is', a_datetime.strftime("%w"), '(Weekday as a number 0-6, 0 is Sunday)')
print('the day is', a_datetime.strftime("%d"), '(Day of month 01-31)')
print('the month is', a_datetime.strftime("%b"), '(Month name, short version)')
print('the month is', a_datetime.strftime("%B"), '(Month name, full version)')
print('the month number is', a_datetime.strftime("%m"), '(Month as a number 01-12)')
print('the year is', a_datetime.strftime("%y"), '(Year, short version, without century)')
print('the year is', a_datetime.strftime("%Y"), '(Year, full version)')
print('the 24-hour is', a_datetime.strftime("%H"), '(Hour 00-23)')
print('the 12-hour is', a_datetime.strftime("%I"), '(Hour 00-12)')
print('the morning/evening is', a_datetime.strftime("%p"), '(AM/PM)')
print('the minute is', a_datetime.strftime("%M"), '(Minute 00-59)')
print('the second is', a_datetime.strftime("%S"), '(Second 00-59)')
print('the microsecond is', a_datetime.strftime("%f"), '(Microsecond 000000-999999)')
print('the timezone offset is', a_datetime.strftime("%z"), '(UTC offset)')
print('the timeszone is set', a_datetime.strftime("%Z"), '(Timezone)')
print('the day number is', a_datetime.strftime("%j"), '(Day number of year 001-366)')
print('the week number is', a_datetime.strftime("%U"), '(Week number of year, Sunday as the first day of week, 00-53)')
print('the week number is also', a_datetime.strftime("%W"), '(Week number of year, Monday as the first day of week, 00-53)')
print('today is', a_datetime.strftime("%c"), '(Local version of date and time)')
print('today is', a_datetime.strftime("%x"), '(Local version of date)')
print('today is', a_datetime.strftime("%X"), '(Local version of time)')

today is Sun (Weekday, short version)
today is Sunday (Weekday, full version)
the weekday is 0 (Weekday as a number 0-6, 0 is Sunday)
the day is 30 (Day of month 01-31)
the month is Sep (Month name, short version)
the month is September (Month name, full version)
the month number is 09 (Month as a number 01-12)
the year is 18 (Year, short version, without century)
the year is 2018 (Year, full version)
the 24-hour is 16 (Hour 00-23)
the 12-hour is 04 (Hour 00-12)
the morning/evening is PM (AM/PM)
the minute is 02 (Minute 00-59)
the second is 29 (Second 00-59)
the microsecond is 534185 (Microsecond 000000-999999)
the timezone offset is  (UTC offset)
the timeszone is set  (Timezone)
the day number is 273 (Day number of year 001-366)
the week number is 39 (Week number of year, Sunday as the first day of week, 00-53)
the week number is also 39 (Week number of year, Monday as the first day of week, 00-53)
today is Sun Sep 30 16:02:29 2018 (Local version of date and time)
today is 09/30/18 (L

In [6]:
# a very common, different, way to use strftime():
print(a_datetime.strftime("today is a %a (Weekday, short version)'"))
print(a_datetime.strftime("today is a %A (Weekday, full version)'"))

today is a Sun (Weekday, short version)'
today is a Sunday (Weekday, full version)'


In [7]:
# we often need to read a time stamp from a string in some formats, e.g.:
timestamp_str_1 = '2018-01-21T15:49:23.3855Z'
timestamp_str_2 = '2018/01/21 15:49:23'
timestamp_str_3 = 'Thu 21/01/18 @ 15:49'
timestamp_str_4 = 'At 15:49 on Thursday, 21/1/18'
timestamp_str_4 = "At 15:49 on Thursday, 21 of January '18"
timestamp_str_4 = "21st of January"
timestamp_str_5 = "Jan 21"
# how do we convert these strings to datetime objects?